In [112]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import FuncFormatter
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [113]:
### Taken from https://flynn.gg/blog/better-matplotlib-charts/
def number_formatter(number, pos=None):
    """Convert a number into a human readable format."""
    magnitude = 0
    while abs(number) >= 1000:
        magnitude += 1
        number /= 1000.0
    return '%.1f%s' % (number, ['', 'K', 'M', 'B', 'T', 'Q'][magnitude])
#ax.yaxis.set_major_formatter(FuncFormatter(number_formatter))

In [114]:
df = pd.read_csv('../compare_loss_by_sector.csv')
df['sector'] = df['short'].apply(lambda x: x.title())
df['loss']*=10**6
df['label'] = df['intervention'].apply(lambda x: '{}%,{}d'.format(x.split('_')[-2],x.split('_')[-1]) if x!='none' else 'base')
df.sort_values(['label','sector'],ascending=False,inplace=True)

In [115]:
colors = dict(zip(['60%','70%','80%','90%'],['#1b9e77','#d95f02','#7570b3','#e7298a']))
markers = dict(zip(['30d','45d','60d'],['o','^','s']))

In [ ]:
plt.figure(figsize=(10,8.5))
ax = plt.gca()
for itv_l in df['label'].unique():
    alpha = 1
    if itv_l != '70%,45d':
        alpha = 0.1
    if itv_l != 'base':
        df[df['label']==itv_l].plot.scatter(x='loss',y='sector',marker=markers[itv_l.split(',')[1]],color=colors[itv_l.split(',')[0]],alpha=alpha,ax=ax,s=100)
df[df['label']=='base'].plot.scatter(x='loss',y='sector',color='k',marker='d',s=100,ax=ax)
band_color = ['#dbdbdb','#bdbdbd']
for y in ax.get_yticks():
    ax.axhspan(y-0.5,y+0.5,color=band_color[y%2],zorder=0,alpha=0.5)

ax.set_ylim([-0.5,20.5])
ax.xaxis.set_major_formatter(FuncFormatter(number_formatter))
plt.tick_params(labelsize=14)
plt.xlabel('Loss in dollars',fontsize=16)
plt.ylabel('Sector',fontsize=16)
plt.legend(handles=[mpatches.Patch(color=colors[k],label=k) for k in colors.keys()] + 
                   [mlines.Line2D([], [], color='k',marker=markers[k],linestyle='None',markersize=10,markerfacecolor='none',label=k) for k in markers.keys()] +
                   [mlines.Line2D([],[],color='k',marker='d',linestyle='None',markersize=10,label='base')],fontsize=14)

# plt.legend(handles=[mpatches.Patch(color=colors[k],label=k) for k in ['70%']] + 
#                    [mlines.Line2D([], [], color='k',marker=markers[k],linestyle='None',markersize=10,markerfacecolor='none',label=k) for k in ['45d']] +
#                    [mlines.Line2D([],[],color='k',marker='d',linestyle='None',markersize=10,label='base')],fontsize=14)

plt.savefig('../../fig/loss_per_sector.png',bbox_inches='tight',dpi=100)